In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content

soup = BeautifulSoup(c, "html.parser")

#all = soup.find_all("div", {"class":"propertyRow"}) 

page_nr = soup.find_all("a", {"class":"Page"})[-1].text

In [2]:
list_features = []

base_url = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="

for page in range(0, int(page_nr)*10, 10):
    print(base_url + str(page) + ".html")
    r = requests.get(base_url + str(page) + ".html", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all("div", {"class":"propertyRow"}) 

    for i in all:
        d = {}
        
        d["Address"] = i.find_all("span", {"class":"propAddressCollapse"})[0].text
        
        try:
            d["Locality"] = i.find_all("span", {"class":"propAddressCollapse"})[1].text
        except:
            d["Locality"] = None
        
        d["Price"] = i.find("h4", {"class":"propPrice"}).text.replace("\n", "").replace(" ", "")

        try:
            d["Beds"] = int(i.find("span", {"class":"infoBed"}).find("b").text)
        except:
            d["Beds"] = None

        try:
            d["Area"] = int(i.find("span", {"class":"infoSqFt"}).find("b").text)
        except:
            d["Area"] = None

        try:
            d["Full Baths"] = int(i.find("span", {"class":"infoValueFullBath"}).find("b").text)
        except:
            d["Full Baths"] = None

        try:
            d["Half Baths"] = int(i.find("span", {"class":"infoValueHalfBath"}).find("b").text)
        except:
            d["Half Baths"] = None

        try:
            for column_group in i.find_all("div", {"class":"columnGroup"}):
                for feature_group, feature_name in zip(column_group.find_all("span", {"class":"featureGroup"}),
                                                   column_group.find_all("span", {"class": "featureName"})):
                    if "Lot Size" in feature_group.text:
                        d["Lot Size"] = feature_name.text
        except:
            d["Lot Size"] = None

        list_features.append(d)

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html


In [3]:
# store the above list in a dataframe

import pandas as pd

df = pd.DataFrame(list_features)
df.to_excel("webscraping_output.xlsx")